In [ ]:
!pip install torchxrayvision
!pip install numpy
!pip install pillow
!pip install pydicom

In [ ]:
import torchxrayvision
from re import L
from pyparsing.helpers import PositionToken
from pandas import Index
#Code is first test on https://github.com/mlmed/torchxrayvision
import torchxrayvision as xrv
import skimage, torch, torchvision
import matplotlib.pyplot as plt
import os
import tabulate
import numpy as np
import pydicom
import cv2 as cv

#added code
from google.colab import drive
from PIL import Image
from os import listdir
from tabulate import tabulate
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from datetime import datetime, date
import json

drive.mount('/content/drive')


In [ ]:

def calculate_age(birth_date, current_date):
    # Calculation
    years = current_date.year - birth_date.year
    months = current_date.month - birth_date.month
    days = current_date.day - birth_date.day

    # Adjust for negative differences
    if days < 0:
        months -= 1
        days += get_days_in_month(birth_date.month, birth_date.year)
    if months < 0:
        years -= 1
        months += 12

    return years, months, days

def get_days_in_month(month, year):
    # Returns the number of days in a given month and year
    if month == 2:  # February
        if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0):
            return 29  # Leap year
        else:
            return 28
    elif month in [4, 6, 9, 11]:  # April, June, September, November
        return 30
    else:
        return 31

def dcm_patient_data(path_file):
  dcm_data = pydicom.dcmread(path_file)
  patient_data = {}
  have_patient_dob = False
  have_acquisition_date = False
  patient_birth_date = ""
  patient_age_now = ""
  acquisition_date = ""
  acquisition_age = ""

  patient_data.update({"Study ID" :str(dcm_data.StudyID)})
  patient_data.update({"Patient ID" :str(dcm_data.PatientID)})
  patient_data.update({"Patient Name" :str(dcm_data.PatientName)})
  patient_data.update({"Patient Sex" :str(dcm_data.PatientSex)})

  if dcm_data.PatientBirthDate == "":
    patient_birth_date = ""
    patient_age_now = ""
  else:
    have_patient_dob = True
    birth_date = datetime.utcfromtimestamp(int(dcm_data.PatientBirthDate))
    if birth_date <= datetime.now():
      age_years, age_months, age_days = calculate_age(birth_date, datetime.now())
      patient_age_now = str(age_years) + " years, " + str(age_months) + " months, " + str(age_days) + " days"
      patient_birth_date = birth_date.strftime('%Y-%m-%d')

  if dcm_data.AcquisitionDate == "":
      acquisition_date = ""
      acquisition_age = ""
  else:
    have_acquisition_date = True
    acquisition_date = datetime.utcfromtimestamp(int(dcm_data.AcquisitionDate))
    patient_acquisition_date = acquisition_date.strftime('%Y-%m-%d')

  if (have_patient_dob == True and have_acquisition_date == True):

    if birth_date <= acquisition_date:
      age_years, age_months, age_days = calculate_age(birth_date, acquisition_date)
      acquisition_age = str(age_years) + " years, " + str(age_months) + " months, " + str(age_days) + " days"


  patient_data.update({"Patient Birth Date" :str(( patient_birth_date))})
  patient_data.update({"Patient Current Age" :str(( patient_age_now))})
  patient_data.update({"Series Number" :str(dcm_data.SeriesNumber)})
  patient_data.update({"Acquisition Number" :str(dcm_data.AcquisitionNumber)})
  patient_data.update({"Acquisition Date" :str( ( patient_acquisition_date))})
  patient_data.update({"Instance Number" :str(dcm_data.InstanceNumber)})
  patient_data.update({"View Position" :str(dcm_data.ViewPosition)})
  patient_data.update({"Patient Orientation" :str(dcm_data.PatientOrientation)})
  patient_data.update({"Patient Age at Time of Acquisition" :str(( acquisition_age))})

  return patient_data

In [ ]:
#takes input as file path to jpg image
#weight_string = "densenet121-res224-mimic_ch" for MIMIC-CXR (MIT)
#weight_string = "densenet121-res224-mimic_nb" for MIMIC-CXR (MIT)
#weight_string = "densenet121-res224-nih" for NIH chest X-ray8

#needs to split into function that takes jpg image and make a function that takes dicom and returns patient infomration- can use dcm_patient_data
def run_model(file_path, weight_string):

  img = skimage.io.imread(file_path)
  #plt.imshow(img);
  img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
  img = img.mean(2)[None, ...] # Make single color channel

  transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
  img = transform(img)
  img = torch.from_numpy(img)

  # Load model and process image
  #model = xrv.models.DenseNet(weights="densenet121-res224-nih") # NIH chest X-ray8
  model = xrv.models.DenseNet(weights=weight_string) # MIMIC-CXR (MIT)
  #model = xrv.models.DenseNet(weights="densenet121-res224-mimic_nb") # MIMIC-CXR (MIT)
  outputs = model(img[None,...]) # or outputs = model.features(img[None,...])
  probability_dictionary = dict(zip(model.pathologies,outputs[0].detach().numpy()))

  model_dict = {}

  #patient_data.update({"Instance Number" :dcm_data.InstanceNumber})
  model_dict.update({"Model Atelectasis":float(probability_dictionary['Atelectasis'])})
  model_dict.update({"Model Cardiomegaly":float(probability_dictionary['Cardiomegaly'])})
  model_dict.update({"Model Consolidation":float(probability_dictionary['Consolidation'])})
  model_dict.update({"Model Edema":float(probability_dictionary['Edema'])})
  model_dict.update({"Model Effusion":float(probability_dictionary['Effusion'])})


  return model_dict


In [ ]:
#takes input as file path to dicom image, and saves two files, image.jpg and scaled_image.jpg
def convert_dcm_to_jpg(file_path):
  ds = pydicom.dcmread(file_path)

  dcm_data = pydicom.dcmread(file_path)
  #print(dcm_data)

  new_image = ds.pixel_array.astype(float)
  scaled_image = (np.maximum(new_image, 0) / new_image.max()) * 255.0
  scaled_image = np.uint8(scaled_image)
  final_image = Image.fromarray(scaled_image)
  final_image.save('image.jpg')

  #read the image
  image = cv.imread("image.jpg")
  #convert the image to RGB (images are read in BGR in OpenCV)
  RGB = cv.cvtColor(image, cv.COLOR_BGR2RGB)
  new_image = Image.fromarray(RGB)
  #plt.imshow(new_image);
  new_image = new_image.resize((1024, 1024))
  new_image.save('scaled_image.jpg')
  return


In [ ]:
#takes in file path to cvs file containing the study tags for each image and the study id for the image you want's data as a string
def actual_dcm_dict(file_path,study_id):
  line_content = []

  with open(file_path) as myFile:
      for num, line in enumerate(myFile, 0):
          if study_id in line:
              line_content = line.split(",")
              #print("the line content is: ", line_content)

  Atelectasis_index = 2
  Cardiomegaly_index = 3
  Consolidation_index = 4
  Edema_index = 5
  Pleural_Effusion_index = 11

  actual_dictionary = {
      'Actual Atelectasis': 0,
      'Actual Cardiomegaly': 0,
      'Actual Consolidation': 0,
      'Actual Edema': 0,
      'Actual Effusion': 0
  }

  if (line_content[Atelectasis_index] == "1.0"): actual_dictionary['Actual Atelectasis'] = 1
  if (line_content[Cardiomegaly_index] == "1.0"): actual_dictionary['Actual Cardiomegaly'] = 1
  if (line_content[Consolidation_index] == "1.0"): actual_dictionary['Actual Consolidation'] = 1
  if (line_content[Edema_index] == "1.0"): actual_dictionary['Actual Edema'] = 1
  if (line_content[Pleural_Effusion_index] == "1.0"): actual_dictionary['Actual Effusion'] = 1

  return  actual_dictionary

In [ ]:
#takes in array of dictionary and returns and creates a json file called output.json containing all entries and returns a json ojbect with all values
def create_json(original_dict):
  new_dict = {}
  out_file = open("output.json", "w")
  for x in original_dict:
    new_dict.update(x)
    new_dict.update(x)
    new_dict.update(x)

  json.dumps(new_dict, indent = 4)

  json.dump(new_dict, out_file, indent = 4)

  out_file.close()

  return json.dumps(new_dict, indent = 4)

In [ ]:
def dcm_to_json(file_path,weight_string,csv_file_path):
#takes input as file path to jpg image
#weight_string = "densenet121-res224-mimic_ch" for MIMIC-CXR (MIT)
#weight_string = "densenet121-res224-mimic_nb" for MIMIC-CXR (MIT)
#weight_string = "densenet121-res224-nih" for NIH chest X-ray8


  convert_dcm_to_jpg(file_path)

  patient_data = dcm_patient_data(file_path)

  actual_tags = actual_dcm_dict(csv_file_path, patient_data["Study ID"] )

  model_output = run_model("scaled_image.jpg",weight_string)

  json_value = create_json([patient_data,model_output,actual_tags])

  return json_value

Functions that can be called on the back end

<br/>

def dcm_to_json(file_path,weight_string,csvfile__path):\
input: file_path is string pointing to a jpg file. weight_string is string of what model weight to use in torchxrayvision's DenseNet model (possible values below). csv_path is string of path to csv file containing tags for mimic dataset images.\
files_created: output.json image.jpg scaled_image.jpg\
output: json containing model outputs for the image, mimic desise tags for the image and patient information

<br/>

def dcm_patient_data(path_file):\
input: path_file is string of the path to a dicom file\
output: dictionary with patient information

<br/>

def run_model(file_path,weight_string):\
input: file_path is string pointing to a jpg file. weight_string is string of what model weight to use in torchxrayvision's DenseNet model\
output: dictionary with model's output

<br/>

def convert_dcm_to_jpg(file_path):\
input: filepath is string to dicom image\
files_created: image.jpg and scaled_image.jpg\
output: Nothing

<br/>

def actual_dcm_dict(file_path,study_id):\
input: file_path is string of path to csv file containing tags for mimic dataset images. study_id is string of id from that study, is in dictionary returnd by dcm_patient_data with key ["Study ID"] \
output: dictionary with tags from mimic dataset for that study

<br/>

def create_json(original_dict):
input: array of dictionaries\
files_created: output.json\
output: a json object containing entries from all dictionaries

<br/>




Usefull wegiht_strings\
wegiht_strings = "densenet121-res224-mimic_ch" for MIMIC-CXR (MIT)\
wegiht_strings = "densenet121-res224-mimic_nb" for MIMIC-CXR (MIT)\
wegiht_strings = "densenet121-res224-nih" for NIH chest X-ray8

In [ ]:
#testing functions


#weights = "densenet121-res224-mimic_ch" for MIMIC-CXR (MIT)
#weights = "densenet121-res224-mimic_nb" for MIMIC-CXR (MIT)
#weights = "densenet121-res224-nih" for NIH chest X-ray8
path = "/content/drive/MyDrive/Shared Capstone/Colab/mimic_images/demo-files"
file_name = "demo2.dcm"
mimix_csv = "/content/drive/MyDrive/Shared Capstone/Colab/mimic_images/mimic-cxr-2.0.0-chexpert.csv"
weights = "densenet121-res224-mimic_ch"
#path = "/content/drive/MyDrive/Shared Capstone/Colab/mimic_images/files/p10/p10000032/s50414267"
#file_name = "02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.dcm"
#file_name = "174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.dcm"

output = dcm_to_json(path + "/" + file_name,weights,mimix_csv )
print(output)

In [ ]:
#test the functions

path = "/content/drive/MyDrive/Shared Capstone/Colab/mimic_images/demo-files"
file_name = "demo2.dcm"
mimix_csv = "/content/drive/MyDrive/Shared Capstone/Colab/mimic_images/mimic-cxr-2.0.0-chexpert.csv"
weights = "densenet121-res224-mimic_ch"
#path = "/content/drive/MyDrive/Shared Capstone/Colab/mimic_images/files/p10/p10000032/s50414267"
#file_name = "02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.dcm"
#file_name = "174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.dcm"

convert_dcm_to_jpg(path + "/" + file_name)

patient_data = dcm_patient_data(path + "/" + file_name)

actual_tags = actual_dcm_dict("/content/drive/MyDrive/Shared Capstone/Colab/mimic_images/mimic-cxr-2.0.0-chexpert.csv", patient_data["Study ID"] )

model_output = run_model("scaled_image.jpg","densenet121-res224-mimic_ch")
#second argument = "densenet121-res224-mimic_ch" for MIMIC-CXR (MIT)
#second argument = "densenet121-res224-mimic_nb" for MIMIC-CXR (MIT)
#second argument = "densenet121-res224-nih" for NIH chest X-ray8


#print("Patient data: " + str(patient_data))
#print ("Model output: " + str(model_output))
#print ("Actual tags: " + str(actual_tags))
json_value = create_json([patient_data,model_output,actual_tags])
print(json_value)